In [1]:
import os
import osr
import glob
import gdal
import numpy as np
import matplotlib.pyplot as plt
import datetime
import h5py
import calendar
# Ignore runtime warning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# (Function 1) Extract data layers from MODIS HDF5 files, filter low quality pixels and write to GeoTiff
# (The function only works for MODIS LST and NDVI data sets!)

def hdf_subdataset_extraction(hdf_files, subdataset_id, band_n):

    # Open the dataset
    global band_ds
    hdf_ds = gdal.Open(hdf_files, gdal.GA_ReadOnly)

    # Loop read data of specified bands from subdataset_id
    size_1dim = gdal.Open(hdf_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly).ReadAsArray().astype(np.int16).shape
    band_array = np.empty([size_1dim[0], size_1dim[1], len(subdataset_id)//2])
    for idn in range(len(subdataset_id)//2):
        band_ds = gdal.Open(hdf_ds.GetSubDatasets()[subdataset_id[idn*2]][0], gdal.GA_ReadOnly)
        band_ds_arr = band_ds.ReadAsArray().astype(np.float32)
        band_ds_arr_qa = gdal.Open(hdf_ds.GetSubDatasets()[subdataset_id[idn*2+1]][0], gdal.GA_ReadOnly)

        # Find if the dataset is MODIS NDVI or MODIS LST
        if len(subdataset_id) > 2: # MODIS LST
            band_ds_arr = band_ds_arr * 0.02
            band_ds_arr_qa = band_ds_arr_qa.ReadAsArray().astype(np.uint8)
            band_ds_arr_qa = band_ds_arr_qa.reshape(-1, 1)
            band_ds_arr_qa_bin = [np.binary_repr(band_ds_arr_qa[x].item(), width=8) for x in range(len(band_ds_arr_qa))]
            ind_qa = np.array([i for i in range(len(band_ds_arr_qa_bin)) if band_ds_arr_qa_bin[i][0] == '0'
                               and band_ds_arr_qa_bin[i][6] == '0'])
        else: # MODIS NDVI
            band_ds_arr = band_ds_arr * 0.0001
            band_ds_arr_qa = band_ds_arr_qa.ReadAsArray().astype(np.uint16)
            band_ds_arr_qa = band_ds_arr_qa.reshape(-1, 1)
            band_ds_arr_qa_bin = [np.binary_repr(band_ds_arr_qa[x].item(), width=16) for x in range(len(band_ds_arr_qa))]
            ind_qa = np.array([i for i in range(len(band_ds_arr_qa_bin)) if band_ds_arr_qa_bin[i][-1] == '0'])

        band_ds_arr_qa_mask = np.arange(band_ds_arr.shape[0]*band_ds_arr.shape[1])
        band_ds_arr_qa_mask = np.isin(band_ds_arr_qa_mask, ind_qa).astype(int)
        band_ds_arr_qa_mask = band_ds_arr_qa_mask.reshape(band_ds_arr.shape[0], band_ds_arr.shape[1])
        band_ds_arr = band_ds_arr * band_ds_arr_qa_mask
        band_ds_arr[np.where(band_ds_arr <= 0)] = np.nan

        # Write into numpy array
        band_array[:, :, idn] = band_ds_arr
        del(band_ds_arr, band_ds_arr_qa, band_ds_arr_qa_bin, ind_qa, band_ds_arr_qa_mask)

    # Build output path
    # band_path = os.path.join(path_modis_op, os.path.basename(os.path.splitext(hdf_files)[0]) + "-ctd" + ".tif")
    # Write raster
    out_ds = gdal.GetDriverByName('MEM').Create('', band_ds.RasterXSize, band_ds.RasterYSize, band_n, #Number of bands
                                  gdal.GDT_Float32)
    out_ds.SetGeoTransform(band_ds.GetGeoTransform())
    out_ds.SetProjection(band_ds.GetProjection())

    # Loop write each band to Geotiff file
    for idb in range(len(subdataset_id)//2):
        out_ds.GetRasterBand(idb+1).WriteArray(band_array[:, :, idb])
        out_ds.GetRasterBand(idb+1).SetNoDataValue(0)
    # out_ds = None  #close dataset to write to disc

    return out_ds

########################################################################################################################

# 0. Input variables
# Specify file paths
# Path of current workspace
path_workspace = '/Users/binfang/Documents/SMAP_CONUS/codes_py'
# Path of source MODIS data
path_modis = '/Users/binfang/Downloads/Processing/HDF'
# # Path of output MODIS data
# path_modis_op = '/Volumes/MyPassport/SMAP_Project/NewData/MODIS/Output'
# Path of MODIS data for SM downscaling model input
path_modis_model = '/Users/binfang/Downloads/Processing/Model_Input'
# Path of SM model output
path_model_op = '/Users/binfang/Downloads/Processing/Model_Output'
# Path of SMAP data
path_smap = '/Volumes/MyPassport/SMAP_Project/NewData/SMAP'
# Path of processed data
path_procdata = '/Users/binfang/Downloads/Processing'
# Path of Land mask
path_lmask = '/Volumes/MyPassport/SMAP_Project/Datasets/Lmask'
lst_folder = '/MYD11A1/'
ndvi_folder = '/MYD13A2/'
subfolders = np.arange(2015, 2019+1, 1)
subfolders = [str(i).zfill(4) for i in subfolders]

# Load in variables
os.chdir(path_workspace)
f = h5py.File("ds_parameters.hdf5", "r")
varname_list = ['lat_world_max', 'lat_world_min', 'lon_world_max', 'lon_world_min',
                'lat_world_ease_9km', 'lon_world_ease_9km', 'lat_world_ease_1km', 'lon_world_ease_1km',
                'lat_world_geo_1km', 'lon_world_geo_1km', 'row_world_ease_1km_from_geo_1km_ind',
                'col_world_ease_1km_from_geo_1km_ind']

for x in range(len(varname_list)):
    var_obj = f[varname_list[x]][()]
    exec(varname_list[x] + '= var_obj')
f.close()

# Generate sequence of string between start and end dates (Year + DOY)
start_date = '2015-04-01'
end_date = '2019-10-31'

start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
delta_date = end_date - start_date

date_seq = []
for i in range(delta_date.days + 1):
    date_str = start_date + datetime.timedelta(days=i)
    date_seq.append(str(date_str.timetuple().tm_year) + str(date_str.timetuple().tm_yday).zfill(3))

# Count how many days for a specific year
yearname = np.linspace(2015, 2019, 5, dtype='int')
monthnum = np.linspace(1, 12, 12, dtype='int')
monthname = np.arange(1, 13)
monthname = [str(i).zfill(2) for i in monthname]

daysofyear = []
for idt in range(len(yearname)):
    if idt == 0:
        f_date = datetime.date(yearname[idt], monthnum[3], 1)
        l_date = datetime.date(yearname[idt], monthnum[-1], 31)
        delta_1y = l_date - f_date
        daysofyear.append(delta_1y.days + 1)
    else:
        f_date = datetime.date(yearname[idt], monthnum[0], 1)
        l_date = datetime.date(yearname[idt], monthnum[-1], 31)
        delta_1y = l_date - f_date
        daysofyear.append(delta_1y.days + 1)

daysofyear = np.asarray(daysofyear)

# Find the indices of each month in the list of days between 2015 - 2018
nlpyear = 1999 # non-leap year
lpyear = 2000 # leap year
daysofmonth_nlp = np.array([calendar.monthrange(nlpyear, x)[1] for x in range(1, len(monthnum)+1)])
ind_nlp = [np.arange(daysofmonth_nlp[0:x].sum(), daysofmonth_nlp[0:x+1].sum()) for x in range(0, len(monthnum))]
daysofmonth_lp = np.array([calendar.monthrange(lpyear, x)[1] for x in range(1, len(monthnum)+1)])
ind_lp = [np.arange(daysofmonth_lp[0:x].sum(), daysofmonth_lp[0:x+1].sum()) for x in range(0, len(monthnum))]
ind_iflpr = np.array([int(calendar.isleap(yearname[x])) for x in range(len(yearname))]) # Find out leap years
# Generate a sequence of the days of months for all years
daysofmonth_seq = np.array([np.tile(daysofmonth_nlp[x], len(yearname)) for x in range(0, len(monthnum))])
daysofmonth_seq[1, :] = daysofmonth_seq[1, :] + ind_iflpr # Add leap days to February

# MODIS data layer information for extraction
# n^th of Layers to be extracted. The information of number of layers can be acquired by function GetSubDatasets()
subdataset_id_lst = [0, 1, 4, 5]  # For MODIS LST data: extract LST_Day_1km, QC_Day, LST_Night_1km, QC_Night
band_n_lst = 2  # For MODIS LST data: save LST_Day_1km, LST_Night_1km
subdataset_id_ndvi = [0, 2] # For MODIS NDVI data: extract NDVI, VI Quality
band_n_ndvi = 1 # For MODIS NDVI data: save NDVI

# Set the boundary coordinates of the map to subset (World)
lat_roi_max = 90
lat_roi_min = -90
lon_roi_max = 180
lon_roi_min = -180

modis_folders = [lst_folder, ndvi_folder]
subdataset_id = [subdataset_id_lst, subdataset_id_ndvi]
band_n = [band_n_lst, band_n_ndvi]
modis_var_names = ['modis_lst_1km', 'modis_ndvi_1km']

# Define target SRS
dst_srs = osr.SpatialReference()
dst_srs.ImportFromEPSG(6933)  # EASE grid projection
dst_wkt = dst_srs.ExportToWkt()
gts = (-17367530.44516138, 1000.89502334956, 0, 7314540.79258289, 0, -1000.89502334956)














In [5]:
########################################################################################################################
# 1. Extract, mosaic and reproject MODIS tile data to EASE grid projection at 1 km
# 1.1 Extract data layers from MODIS HDF5 files and write to GeoTiff 
# Mosaic the list of MODIS geotiff files and reproject to lat/lon projection

modis_mat_ease_1day_init = np.empty([len(lat_world_ease_1km), len(lon_world_ease_1km)], dtype='float32')
modis_mat_ease_1day_init[:] = np.nan

for ifo in [1]: #range(len(modis_folders)): # MODIS LST and NDVI subfolders

    for iyr in range(4, len(subfolders)):

        for imo in range(9, len(monthname)):

            path_month = path_modis + modis_folders[ifo] + subfolders[iyr] + '/' + monthname[imo]
            os.chdir(path_month)
            hdf_files = sorted(glob.glob('*.hdf'))

            if len(hdf_files) != 0:

                hdf_file_name = [hdf_files[x].split('.')[1] for x in range(len(hdf_files))]
                hdf_file_name_unique = sorted(list(set(hdf_file_name)))

                # Group the MODIS tile files by each day
                for idt in range(len(hdf_file_name_unique)):

                    hdf_files_toBuild_ind = [hdf_files.index(i) for i in hdf_files if hdf_file_name_unique[idt] in i]
                    hdf_files_toBuild = [hdf_files[i] for i in hdf_files_toBuild_ind]

                    hdf_files_list = []
                    for idf in range(len(hdf_files_toBuild)):
                        extr_file = hdf_subdataset_extraction(path_month + '/' + hdf_files_toBuild[idf],
                                                              subdataset_id[ifo], band_n[ifo])
                        hdf_files_list.append(extr_file)  # Append the processed hdf to the file list being merged
                        print(hdf_files_toBuild[idf])  # Print the file being processed
                        del(extr_file)

                    # Open file and warp the target raster dimensions and geotransform
                    out_ds = gdal.Warp('', hdf_files_list, format='MEM', outputBounds=[-180, -90, 180, 90], xRes=0.01, yRes=0.01,
                                       dstSRS='EPSG:4326', warpOptions=['SKIP_NOSOURCE=YES'], errorThreshold=0,
                                       resampleAlg=gdal.GRA_NearestNeighbour)

                    modis_mat = out_ds.ReadAsArray()
                    modis_mat[np.where(modis_mat <= 0)] = np.nan
                    modis_mat = np.atleast_3d(modis_mat)
                    # For MODIS LST data layers
                    if modis_mat.shape[0] == 2:
                        modis_mat = np.transpose(modis_mat, (1, 2, 0))
                    else:
                        pass

                    # Create initial EASE grid projection matrices at 1 km
                    modis_mat_ease = \
                        np.empty([len(lat_world_ease_1km), len(lon_world_ease_1km), modis_mat.shape[2]], dtype='float32')
                    modis_mat_ease[:] = np.nan

                    for idm in range(modis_mat.shape[2]):
                        modis_mat_ease_1day = np.copy(modis_mat_ease_1day_init)
                        modis_mat_1day = modis_mat[:, :, idm]
                        modis_mat_ease_1day = np.array \
                            ([np.nanmean(modis_mat_1day[row_world_ease_1km_from_geo_1km_ind[x], :], axis=0)
                              for x in range(len(lat_world_ease_1km))])
                        modis_mat_ease_1day = np.array \
                            ([np.nanmean(modis_mat_ease_1day[:, col_world_ease_1km_from_geo_1km_ind[y]], axis=1)
                              for y in range(len(lon_world_ease_1km))])
                        modis_mat_ease_1day = np.fliplr(np.rot90(modis_mat_ease_1day, 3))
                        modis_mat_ease[:, :, idm] = modis_mat_ease_1day
                        del(modis_mat_ease_1day, modis_mat_1day)

                    del(modis_mat, out_ds, hdf_files_list, hdf_files_toBuild, hdf_files_toBuild_ind)


                    # 1.4 Save the daily MODIS LST/NDVI data to Geotiff files
                    # Build output path
                    path_writefile = path_modis_model + modis_folders[ifo] + subfolders[iyr]
                    # os.chdir(path_modis_model + modis_folders[ifo] + subfolders[iyr])

                    # Create a raster of EASE grid projection at 1 km resolution
                    out_ds_tiff = gdal.GetDriverByName('GTiff').Create\
                        (path_writefile + '/' + modis_var_names[ifo] + '_' + hdf_file_name_unique[idt][1:] + '.tif',
                         len(lon_world_ease_1km), len(lat_world_ease_1km), band_n[ifo],  # Number of bands
                         gdal.GDT_Float32, ['COMPRESS=LZW', 'TILED=YES'])
                    out_ds_tiff.SetGeoTransform(gts)
                    out_ds_tiff.SetProjection(dst_wkt)

                    # Loop write each band to Geotiff file
                    for idl in range(band_n[ifo]):
                        out_ds_tiff.GetRasterBand(idl + 1).WriteArray(modis_mat_ease[:, :, idl])
                        out_ds_tiff.GetRasterBand(idl + 1).SetNoDataValue(0)
                    out_ds_tiff = None  # close dataset to write to disc

                    print(modis_var_names[ifo] + '_' + hdf_file_name_unique[idt][1:])
                    del(modis_mat_ease)

            else:
                pass

            del (hdf_files, path_month)

MYD13A2.A2019281.h00v08.006.2019304050442.hdf
MYD13A2.A2019281.h00v09.006.2019304045830.hdf
MYD13A2.A2019281.h00v10.006.2019304045751.hdf
MYD13A2.A2019281.h01v07.006.2019304050437.hdf
MYD13A2.A2019281.h01v08.006.2019304045248.hdf
MYD13A2.A2019281.h01v09.006.2019304045715.hdf
MYD13A2.A2019281.h01v10.006.2019304050513.hdf
MYD13A2.A2019281.h01v11.006.2019304044845.hdf
MYD13A2.A2019281.h02v06.006.2019304045811.hdf
MYD13A2.A2019281.h02v08.006.2019304045714.hdf
MYD13A2.A2019281.h02v09.006.2019304050250.hdf
MYD13A2.A2019281.h02v10.006.2019304050501.hdf
MYD13A2.A2019281.h02v11.006.2019304045817.hdf
MYD13A2.A2019281.h03v06.006.2019304045629.hdf
MYD13A2.A2019281.h03v07.006.2019304052009.hdf
MYD13A2.A2019281.h03v09.006.2019304045536.hdf
MYD13A2.A2019281.h03v10.006.2019304050245.hdf
MYD13A2.A2019281.h03v11.006.2019304050218.hdf
MYD13A2.A2019281.h04v09.006.2019304050512.hdf
MYD13A2.A2019281.h04v10.006.2019304045814.hdf
MYD13A2.A2019281.h04v11.006.2019304045644.hdf
MYD13A2.A2019281.h05v10.006.201930

MYD13A2.A2019281.h21v13.006.2019304050453.hdf
MYD13A2.A2019281.h22v01.006.2019304045636.hdf
MYD13A2.A2019281.h22v02.006.2019304050521.hdf
MYD13A2.A2019281.h22v03.006.2019304045832.hdf
MYD13A2.A2019281.h22v04.006.2019304050354.hdf
MYD13A2.A2019281.h22v05.006.2019304050327.hdf
MYD13A2.A2019281.h22v06.006.2019304045348.hdf
MYD13A2.A2019281.h22v07.006.2019304044702.hdf
MYD13A2.A2019281.h22v08.006.2019304050138.hdf
MYD13A2.A2019281.h22v09.006.2019304044752.hdf
MYD13A2.A2019281.h22v10.006.2019304045326.hdf
MYD13A2.A2019281.h22v11.006.2019304050726.hdf
MYD13A2.A2019281.h22v13.006.2019304050331.hdf
MYD13A2.A2019281.h22v14.006.2019304050826.hdf
MYD13A2.A2019281.h23v01.006.2019304045651.hdf
MYD13A2.A2019281.h23v02.006.2019304051545.hdf
MYD13A2.A2019281.h23v03.006.2019304045354.hdf
MYD13A2.A2019281.h23v04.006.2019304045805.hdf
MYD13A2.A2019281.h23v05.006.2019304050326.hdf
MYD13A2.A2019281.h23v06.006.2019304045423.hdf
MYD13A2.A2019281.h23v07.006.2019304045055.hdf
MYD13A2.A2019281.h23v08.006.201930

MYD13A2.A2019297.h11v09.006.2019316202053.hdf
MYD13A2.A2019297.h11v10.006.2019316202952.hdf
MYD13A2.A2019297.h11v11.006.2019316204026.hdf
MYD13A2.A2019297.h11v12.006.2019316202111.hdf
MYD13A2.A2019297.h12v01.006.2019316222058.hdf
MYD13A2.A2019297.h12v02.006.2019316202243.hdf
MYD13A2.A2019297.h12v03.006.2019316202234.hdf
MYD13A2.A2019297.h12v04.006.2019316203524.hdf
MYD13A2.A2019297.h12v05.006.2019316203304.hdf
MYD13A2.A2019297.h12v07.006.2019316204332.hdf
MYD13A2.A2019297.h12v08.006.2019316204313.hdf
MYD13A2.A2019297.h12v09.006.2019316202240.hdf
MYD13A2.A2019297.h12v10.006.2019316203450.hdf
MYD13A2.A2019297.h12v11.006.2019316224133.hdf
MYD13A2.A2019297.h12v12.006.2019316202453.hdf
MYD13A2.A2019297.h12v13.006.2019316204931.hdf
MYD13A2.A2019297.h13v01.006.2019316204105.hdf
MYD13A2.A2019297.h13v02.006.2019316202559.hdf
MYD13A2.A2019297.h13v03.006.2019316203246.hdf
MYD13A2.A2019297.h13v04.006.2019316202903.hdf
MYD13A2.A2019297.h13v08.006.2019316202220.hdf
MYD13A2.A2019297.h13v09.006.201931

MYD13A2.A2019297.h28v08.006.2019316202327.hdf
MYD13A2.A2019297.h28v09.006.2019316202624.hdf
MYD13A2.A2019297.h28v10.006.2019316203644.hdf
MYD13A2.A2019297.h28v11.006.2019316202640.hdf
MYD13A2.A2019297.h28v12.006.2019316203015.hdf
MYD13A2.A2019297.h28v13.006.2019316203526.hdf
MYD13A2.A2019297.h28v14.006.2019316202306.hdf
MYD13A2.A2019297.h29v03.006.2019316202151.hdf
MYD13A2.A2019297.h29v05.006.2019316203749.hdf
MYD13A2.A2019297.h29v06.006.2019316204224.hdf
MYD13A2.A2019297.h29v07.006.2019316203605.hdf
MYD13A2.A2019297.h29v08.006.2019316203743.hdf
MYD13A2.A2019297.h29v09.006.2019316210352.hdf
MYD13A2.A2019297.h29v10.006.2019316203052.hdf
MYD13A2.A2019297.h29v11.006.2019316203620.hdf
MYD13A2.A2019297.h29v12.006.2019316203331.hdf
MYD13A2.A2019297.h29v13.006.2019316203012.hdf
MYD13A2.A2019297.h30v05.006.2019316203024.hdf
MYD13A2.A2019297.h30v06.006.2019316203030.hdf
MYD13A2.A2019297.h30v07.006.2019316203018.hdf
MYD13A2.A2019297.h30v08.006.2019316203036.hdf
MYD13A2.A2019297.h30v09.006.201931

FileNotFoundError: [Errno 2] No such file or directory: '/Users/binfang/Downloads/Processing/HDF/MYD13A2/2019/11'

In [3]:
########################################################################################################################
# 2. Process SMAP enhanced L2 radiometer half-orbit SM 9 km data

matsize_smap_1day = [len(lat_world_ease_9km), len(lon_world_ease_9km)]
smap_mat_init_1day = np.empty(matsize_smap_1day, dtype='float32')
smap_mat_init_1day[:] = np.nan


for iyr in range(4, len(daysofyear)):

    os.chdir(path_smap + '/' + str(yearname[iyr]))
    smap_files_year = sorted(glob.glob('*.h5'))

    # Group SMAP data by month
    for imo in range(len(monthnum)):

        os.chdir(path_smap + '/' + str(yearname[iyr]))
        smap_files_group_1month = [smap_files_year.index(i) for i in smap_files_year if str(yearname[iyr]) + monthname[imo] in i]

        # Process each month
        if len(smap_files_group_1month) != 0:
            smap_files_month = [smap_files_year[smap_files_group_1month[i]] for i in range(len(smap_files_group_1month))]

            # Create initial empty matrices for monthly SMAP final output data
            matsize_smap = [matsize_smap_1day[0], matsize_smap_1day[1], daysofmonth_seq[imo, iyr]]
            smap_mat_month_am = np.empty(matsize_smap, dtype='float32')
            smap_mat_month_am[:] = np.nan
            smap_mat_month_pm = np.copy(smap_mat_month_am)

            # Extract SMAP data layers and rebind to daily
            for idt in range(daysofmonth_seq[imo, iyr]):
                smap_files_group_1day = [smap_files_month.index(i) for i in smap_files_month if
                                         str(yearname[iyr]) + monthname[imo] + str(idt+1).zfill(2) in i]
                smap_files_1day = [smap_files_month[smap_files_group_1day[i]] for i in
                                    range(len(smap_files_group_1day))]
                smap_files_group_1day_am = [smap_files_1day.index(i) for i in smap_files_1day if
                                         'D_' + str(yearname[iyr]) + monthname[imo] + str(idt+1).zfill(2) in i]
                smap_files_group_1day_pm = [smap_files_1day.index(i) for i in smap_files_1day if
                                         'A_' + str(yearname[iyr]) + monthname[imo] + str(idt+1).zfill(2) in i]
                smap_mat_group_1day = \
                    np.empty([matsize_smap_1day[0], matsize_smap_1day[1], len(smap_files_group_1day)], dtype='float32')
                smap_mat_group_1day[:] = np.nan

                # Read swath files within a day and stack
                for ife in range(len(smap_files_1day)):
                    smap_mat_1file = np.copy(smap_mat_init_1day)
                    fe_smap = h5py.File(smap_files_1day[ife], "r")
                    group_list_smap = list(fe_smap.keys())
                    smap_data_group = fe_smap[group_list_smap[1]]
                    varname_list_smap = list(smap_data_group.keys())
                    # Extract variables
                    col_ind = smap_data_group[varname_list_smap[41]][()]
                    row_ind = smap_data_group[varname_list_smap[10]][()]
                    sm_flag = smap_data_group[varname_list_smap[20]][()]
                    sm = smap_data_group[varname_list_smap[25]][()]
                    sm[np.where(sm == -9999)] = np.nan
                    sm[np.where((sm_flag == 7) & (sm_flag == 15))] = np.nan # Refer to the results of np.binary_repr

                    smap_mat_1file[row_ind, col_ind] = sm
                    smap_mat_group_1day[:, :, ife] = smap_mat_1file
                    print(smap_files_1day[ife])
                    fe_smap.close()

                    del(smap_mat_1file, fe_smap, group_list_smap, smap_data_group, varname_list_smap, col_ind, row_ind,
                        sm_flag, sm)

                smap_mat_1day_am = np.nanmean(smap_mat_group_1day[:, :, smap_files_group_1day_am], axis=2)
                smap_mat_1day_pm = np.nanmean(smap_mat_group_1day[:, :, smap_files_group_1day_pm], axis=2)
                # plt.imshow(np.nanmean(np.concatenate((np.atleast_3d(smap_mat_1day_am),
                # np.atleast_3d(smap_mat_1day_pm)), axis=2), axis=2))
                del(smap_mat_group_1day, smap_files_group_1day, smap_files_1day)

                smap_mat_month_am[:, :, idt] = smap_mat_1day_am
                smap_mat_month_pm[:, :, idt] = smap_mat_1day_pm
                del(smap_mat_1day_am, smap_mat_1day_pm)

            # Save file
            os.chdir(path_procdata)
            var_name = ['smap_sm_9km_am_' + str(yearname[iyr]) + monthname[imo],
                        'smap_sm_9km_pm_' + str(yearname[iyr]) + monthname[imo]]
            data_name = ['smap_mat_month_am', 'smap_mat_month_pm']

            with h5py.File('smap_sm_9km_' + str(yearname[iyr]) + monthname[imo] + '.hdf5', 'w') as f:
                for idv in range(len(var_name)):
                    f.create_dataset(var_name[idv], data=eval(data_name[idv]))
            f.close()
            del(smap_mat_month_am, smap_mat_month_pm)

        else:
            pass


SMAP_L2_SM_P_E_20920_D_20190101T000302_R16510_001.h5
SMAP_L2_SM_P_E_20921_A_20190101T005217_R16510_001.h5
SMAP_L2_SM_P_E_20921_D_20190101T014132_R16510_001.h5
SMAP_L2_SM_P_E_20922_A_20190101T023043_R16510_001.h5
SMAP_L2_SM_P_E_20922_D_20190101T031958_R16510_001.h5
SMAP_L2_SM_P_E_20923_A_20190101T040913_R16510_001.h5
SMAP_L2_SM_P_E_20923_D_20190101T045827_R16510_001.h5
SMAP_L2_SM_P_E_20924_A_20190101T054941_R16510_001.h5
SMAP_L2_SM_P_E_20924_D_20190101T063653_R16510_001.h5
SMAP_L2_SM_P_E_20925_A_20190101T072608_R16510_001.h5
SMAP_L2_SM_P_E_20925_D_20190101T084242_R16510_001.h5
SMAP_L2_SM_P_E_20926_A_20190101T090433_R16510_001.h5
SMAP_L2_SM_P_E_20926_D_20190101T095348_R16510_001.h5
SMAP_L2_SM_P_E_20927_A_20190101T104303_R16510_001.h5
SMAP_L2_SM_P_E_20927_D_20190101T113214_R16510_001.h5
SMAP_L2_SM_P_E_20928_A_20190101T122129_R16510_001.h5
SMAP_L2_SM_P_E_20928_D_20190101T131044_R16510_001.h5
SMAP_L2_SM_P_E_20929_A_20190101T135958_R16510_001.h5
SMAP_L2_SM_P_E_20929_D_20190101T144909_R16510_

SMAP_L2_SM_P_E_20998_A_20190106T071345_R16510_001.h5
SMAP_L2_SM_P_E_20998_D_20190106T080300_R16510_001.h5
SMAP_L2_SM_P_E_20999_A_20190106T085215_R16510_001.h5
SMAP_L2_SM_P_E_20999_D_20190106T094125_R16510_001.h5
SMAP_L2_SM_P_E_21000_A_20190106T103040_R16510_001.h5
SMAP_L2_SM_P_E_21000_D_20190106T111955_R16510_001.h5
SMAP_L2_SM_P_E_21001_A_20190106T120910_R16510_001.h5
SMAP_L2_SM_P_E_21001_D_20190106T125821_R16510_001.h5
SMAP_L2_SM_P_E_21002_A_20190106T134736_R16510_001.h5
SMAP_L2_SM_P_E_21002_D_20190106T143650_R16510_001.h5
SMAP_L2_SM_P_E_21003_A_20190106T152605_R16510_001.h5
SMAP_L2_SM_P_E_21003_D_20190106T161516_R16510_001.h5
SMAP_L2_SM_P_E_21004_A_20190106T170431_R16510_001.h5
SMAP_L2_SM_P_E_21004_D_20190106T175346_R16510_001.h5
SMAP_L2_SM_P_E_21005_A_20190106T184301_R16510_001.h5
SMAP_L2_SM_P_E_21005_D_20190106T193211_R16510_001.h5
SMAP_L2_SM_P_E_21006_A_20190106T202126_R16510_001.h5
SMAP_L2_SM_P_E_21006_D_20190106T211041_R16510_001.h5
SMAP_L2_SM_P_E_21007_A_20190106T215952_R16510_

SMAP_L2_SM_P_E_21075_D_20190111T142427_R16510_001.h5
SMAP_L2_SM_P_E_21076_A_20190111T151342_R16510_001.h5
SMAP_L2_SM_P_E_21076_D_20190111T160257_R16510_001.h5
SMAP_L2_SM_P_E_21077_A_20190111T165208_R16510_001.h5
SMAP_L2_SM_P_E_21077_D_20190111T174123_R16510_001.h5
SMAP_L2_SM_P_E_21078_A_20190111T183038_R16510_001.h5
SMAP_L2_SM_P_E_21078_D_20190111T191952_R16510_001.h5
SMAP_L2_SM_P_E_21079_A_20190111T200903_R16510_001.h5
SMAP_L2_SM_P_E_21079_D_20190111T205818_R16510_001.h5
SMAP_L2_SM_P_E_21080_A_20190111T214733_R16510_001.h5
SMAP_L2_SM_P_E_21080_D_20190111T223648_R16510_001.h5
SMAP_L2_SM_P_E_21081_A_20190111T232558_R16510_001.h5
SMAP_L2_SM_P_E_21081_D_20190112T001513_R16510_001.h5
SMAP_L2_SM_P_E_21082_A_20190112T010428_R16510_001.h5
SMAP_L2_SM_P_E_21082_D_20190112T015343_R16510_001.h5
SMAP_L2_SM_P_E_21083_A_20190112T024254_R16510_001.h5
SMAP_L2_SM_P_E_21083_D_20190112T033209_R16510_001.h5
SMAP_L2_SM_P_E_21084_A_20190112T042124_R16510_001.h5
SMAP_L2_SM_P_E_21084_D_20190112T051038_R16510_

SMAP_L2_SM_P_E_21154_D_20190117T000250_R16510_001.h5
SMAP_L2_SM_P_E_21155_A_20190117T005205_R16510_001.h5
SMAP_L2_SM_P_E_21155_D_20190117T014120_R16510_001.h5
SMAP_L2_SM_P_E_21156_A_20190117T023035_R16510_001.h5
SMAP_L2_SM_P_E_21156_D_20190117T031946_R16510_001.h5
SMAP_L2_SM_P_E_21157_A_20190117T040901_R16510_001.h5
SMAP_L2_SM_P_E_21157_D_20190117T045815_R16510_001.h5
SMAP_L2_SM_P_E_21158_A_20190117T054730_R16510_001.h5
SMAP_L2_SM_P_E_21158_D_20190117T063641_R16510_001.h5
SMAP_L2_SM_P_E_21159_A_20190117T072556_R16510_001.h5
SMAP_L2_SM_P_E_21159_D_20190117T081511_R16510_001.h5
SMAP_L2_SM_P_E_21160_A_20190117T090426_R16510_001.h5
SMAP_L2_SM_P_E_21160_D_20190117T095336_R16510_001.h5
SMAP_L2_SM_P_E_21161_A_20190117T104251_R16510_001.h5
SMAP_L2_SM_P_E_21161_D_20190117T113206_R16510_001.h5
SMAP_L2_SM_P_E_21162_A_20190117T122121_R16510_001.h5
SMAP_L2_SM_P_E_21162_D_20190117T131032_R16510_001.h5
SMAP_L2_SM_P_E_21163_A_20190117T135947_R16510_001.h5
SMAP_L2_SM_P_E_21163_D_20190117T144901_R16510_

SMAP_L2_SM_P_E_21232_A_20190122T071333_R16510_001.h5
SMAP_L2_SM_P_E_21232_D_20190122T080248_R16510_001.h5
SMAP_L2_SM_P_E_21233_A_20190122T085203_R16510_001.h5
SMAP_L2_SM_P_E_21233_D_20190122T094117_R16510_001.h5
SMAP_L2_SM_P_E_21234_A_20190122T103028_R16510_001.h5
SMAP_L2_SM_P_E_21234_D_20190122T111943_R16510_001.h5
SMAP_L2_SM_P_E_21235_A_20190122T120858_R16510_001.h5
SMAP_L2_SM_P_E_21235_D_20190122T125813_R16510_001.h5
SMAP_L2_SM_P_E_21236_A_20190122T134723_R16510_001.h5
SMAP_L2_SM_P_E_21236_D_20190122T143638_R16510_001.h5
SMAP_L2_SM_P_E_21237_A_20190122T152553_R16510_001.h5
SMAP_L2_SM_P_E_21237_D_20190122T161508_R16510_001.h5
SMAP_L2_SM_P_E_21238_A_20190122T170419_R16510_001.h5
SMAP_L2_SM_P_E_21238_D_20190122T175334_R16510_001.h5
SMAP_L2_SM_P_E_21239_A_20190122T184249_R16510_001.h5
SMAP_L2_SM_P_E_21239_D_20190122T193203_R16510_001.h5
SMAP_L2_SM_P_E_21240_A_20190122T202114_R16510_001.h5
SMAP_L2_SM_P_E_21240_D_20190122T211029_R16510_001.h5
SMAP_L2_SM_P_E_21241_A_20190122T215944_R16510_

...

SMAP_L2_SM_P_E_25016_D_20191008T014049_R16510_001.h5
SMAP_L2_SM_P_E_25017_A_20191008T023004_R16510_001.h5
SMAP_L2_SM_P_E_25017_D_20191008T031915_R16510_001.h5
SMAP_L2_SM_P_E_25018_A_20191008T040830_R16510_001.h5
SMAP_L2_SM_P_E_25018_D_20191008T045745_R16510_001.h5
SMAP_L2_SM_P_E_25019_A_20191008T054700_R16510_001.h5
SMAP_L2_SM_P_E_25019_D_20191008T063610_R16510_001.h5
SMAP_L2_SM_P_E_25020_A_20191008T072525_R16510_001.h5
SMAP_L2_SM_P_E_25020_D_20191008T081440_R16510_001.h5
SMAP_L2_SM_P_E_25021_A_20191008T090355_R16510_001.h5
SMAP_L2_SM_P_E_25021_D_20191008T095306_R16510_001.h5
SMAP_L2_SM_P_E_25022_A_20191008T104221_R16510_001.h5
SMAP_L2_SM_P_E_25022_D_20191008T113135_R16510_001.h5
SMAP_L2_SM_P_E_25023_A_20191008T122050_R16510_001.h5
SMAP_L2_SM_P_E_25023_D_20191008T131001_R16510_001.h5
SMAP_L2_SM_P_E_25024_A_20191008T135916_R16510_001.h5
SMAP_L2_SM_P_E_25024_D_20191008T144831_R16510_001.h5
SMAP_L2_SM_P_E_25025_A_20191008T153746_R16510_001.h5
SMAP_L2_SM_P_E_25025_D_20191008T162656_R16510_

SMAP_L2_SM_P_E_25096_A_20191013T120827_R16510_003.h5
SMAP_L2_SM_P_E_25096_D_20191013T125742_R16510_003.h5
SMAP_L2_SM_P_E_25097_A_20191013T134652_R16510_003.h5
SMAP_L2_SM_P_E_25097_D_20191013T143607_R16510_003.h5
SMAP_L2_SM_P_E_25098_A_20191013T152522_R16510_002.h5
SMAP_L2_SM_P_E_25098_D_20191013T161437_R16510_002.h5
SMAP_L2_SM_P_E_25099_A_20191013T170348_R16510_003.h5
SMAP_L2_SM_P_E_25099_D_20191013T175303_R16510_002.h5
SMAP_L2_SM_P_E_25100_A_20191013T184217_R16510_003.h5
SMAP_L2_SM_P_E_25100_D_20191013T193128_R16510_002.h5
SMAP_L2_SM_P_E_25101_A_20191013T202043_R16510_002.h5
SMAP_L2_SM_P_E_25101_D_20191013T210958_R16510_003.h5
SMAP_L2_SM_P_E_25102_A_20191013T215913_R16510_003.h5
SMAP_L2_SM_P_E_25102_D_20191013T224823_R16510_003.h5
SMAP_L2_SM_P_E_25103_A_20191013T233738_R16510_002.h5
SMAP_L2_SM_P_E_25103_D_20191014T002653_R16510_002.h5
SMAP_L2_SM_P_E_25104_A_20191014T011608_R16510_001.h5
SMAP_L2_SM_P_E_25104_D_20191014T020519_R16510_001.h5
SMAP_L2_SM_P_E_25105_A_20191014T025434_R16510_

SMAP_L2_SM_P_E_25176_A_20191018T232515_R16510_001.h5
SMAP_L2_SM_P_E_25176_D_20191019T001430_R16510_001.h5
SMAP_L2_SM_P_E_25177_A_20191019T010344_R16510_001.h5
SMAP_L2_SM_P_E_25177_D_20191019T015259_R16510_001.h5
SMAP_L2_SM_P_E_25178_A_20191019T024210_R16510_001.h5
SMAP_L2_SM_P_E_25178_D_20191019T033125_R16510_001.h5
SMAP_L2_SM_P_E_25179_A_20191019T042040_R16510_001.h5
SMAP_L2_SM_P_E_25179_D_20191019T050955_R16510_001.h5
SMAP_L2_SM_P_E_25180_A_20191019T055905_R16510_001.h5
SMAP_L2_SM_P_E_25180_D_20191019T064820_R16510_001.h5
SMAP_L2_SM_P_E_25181_A_20191019T073735_R16510_001.h5
SMAP_L2_SM_P_E_25182_A_20191019T091601_R16510_001.h5
SMAP_L2_SM_P_E_25182_D_20191019T100516_R16510_001.h5
SMAP_L2_SM_P_E_25183_A_20191019T105430_R16510_001.h5
SMAP_L2_SM_P_E_25183_D_20191019T114345_R16510_001.h5
SMAP_L2_SM_P_E_25184_A_20191019T123256_R16510_001.h5
SMAP_L2_SM_P_E_25185_A_20191019T141126_R16510_001.h5
SMAP_L2_SM_P_E_25185_D_20191019T150041_R16510_001.h5
SMAP_L2_SM_P_E_25186_A_20191019T154951_R16510_

SMAP_L2_SM_P_E_25259_A_20191024T153728_R16510_001.h5
SMAP_L2_SM_P_E_25259_D_20191024T162643_R16510_001.h5
SMAP_L2_SM_P_E_25260_A_20191024T171558_R16510_001.h5
SMAP_L2_SM_P_E_25260_D_20191024T180512_R16510_001.h5
SMAP_L2_SM_P_E_25261_A_20191024T185427_R16510_001.h5
SMAP_L2_SM_P_E_25261_D_20191024T194338_R16510_001.h5
SMAP_L2_SM_P_E_25262_A_20191024T203253_R16510_001.h5
SMAP_L2_SM_P_E_25262_D_20191024T212208_R16510_001.h5
SMAP_L2_SM_P_E_25263_A_20191024T221119_R16510_001.h5
SMAP_L2_SM_P_E_25263_D_20191024T230033_R16510_001.h5
SMAP_L2_SM_P_E_25264_A_20191024T234948_R16510_001.h5
SMAP_L2_SM_P_E_25264_D_20191025T003903_R16510_001.h5
SMAP_L2_SM_P_E_25265_A_20191025T012818_R16510_001.h5
SMAP_L2_SM_P_E_25265_D_20191025T021729_R16510_001.h5
SMAP_L2_SM_P_E_25266_A_20191025T030644_R16510_001.h5
SMAP_L2_SM_P_E_25266_D_20191025T035558_R16510_001.h5
SMAP_L2_SM_P_E_25267_A_20191025T044513_R16510_001.h5
SMAP_L2_SM_P_E_25267_D_20191025T053424_R16510_001.h5
SMAP_L2_SM_P_E_25268_A_20191025T062339_R16510_

SMAP_L2_SM_P_E_25338_D_20191030T020505_R16510_001.h5
SMAP_L2_SM_P_E_25339_A_20191030T025420_R16510_001.h5
SMAP_L2_SM_P_E_25339_D_20191030T034335_R16510_001.h5
SMAP_L2_SM_P_E_25340_A_20191030T043246_R16510_001.h5
SMAP_L2_SM_P_E_25340_D_20191030T052201_R16510_001.h5
SMAP_L2_SM_P_E_25341_A_20191030T061116_R16510_001.h5
SMAP_L2_SM_P_E_25341_D_20191030T070030_R16510_001.h5
SMAP_L2_SM_P_E_25342_A_20191030T074941_R16510_001.h5
SMAP_L2_SM_P_E_25342_D_20191030T083856_R16510_001.h5
SMAP_L2_SM_P_E_25343_A_20191030T092811_R16510_001.h5
SMAP_L2_SM_P_E_25343_D_20191030T101726_R16510_001.h5
SMAP_L2_SM_P_E_25344_A_20191030T110636_R16510_001.h5
SMAP_L2_SM_P_E_25344_D_20191030T115551_R16510_001.h5
SMAP_L2_SM_P_E_25345_A_20191030T124506_R16510_001.h5
SMAP_L2_SM_P_E_25345_D_20191030T133421_R16510_001.h5
SMAP_L2_SM_P_E_25346_A_20191030T142332_R16510_001.h5
SMAP_L2_SM_P_E_25346_D_20191030T151247_R16510_001.h5
SMAP_L2_SM_P_E_25347_A_20191030T160201_R16510_001.h5
SMAP_L2_SM_P_E_25347_D_20191030T165116_R16510_